In [109]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'TLMN'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [110]:
from Base.TLMN.env import __ACTIONS__

@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions = np.where(validActions == 1)[0]

    actionTypes = np.zeros_like(validActions)
    for i in range(len(validActions)):
        actionTypes[i] = __ACTIONS__[validActions[i]][0]

    actionType1 = validActions[np.where(actionTypes==1)[0]]
    actionType2 = validActions[np.where(actionTypes==2)[0]]
    actionType3 = validActions[np.where(actionTypes==3)[0]]
    actionType4 = validActions[np.where(actionTypes==4)[0]]
    actionTypes5_13 = validActions[np.where((actionTypes>=5) & (actionTypes<=13))[0]]
    actionType14 = validActions[np.where(actionTypes==14)[0]]
    actionType15 = validActions[np.where(actionTypes==15)[0]]

    if len(actionTypes5_13) > 0:
        return actionTypes5_13[0], per
    
    if len(actionType3) > 0:
        return actionType3[0], per
    
    if len(actionType2) > 0:
        return actionType2[0], per
    
    if len(actionType1) > 0:
        return actionType1[0], per
    
    if len(actionType14) > 0:
        return actionType14[0], per
    
    if len(actionType15) > 0:
        return actionType15[0], per
    
    if len(actionType4) > 0:
        return actionType4[0], per
    
    action = validActions[np.random.randint(len(validActions))]
    return action, per

In [111]:
win, per = numba_main_2(player, 1000, np.array([]), 1)
win

420